# 1. Import Libraries

In [3]:
#!pip install underthesea
import numpy as np
import pandas as pd
import string
string.punctuation

import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

# 2. Load Data

In [4]:
file_path = '/kaggle/input/vietnamese-online-news-dataset/news_dataset.json'

# Đọc file JSON thành DataFrame
df = pd.read_json(file_path)

# Hiển thị dữ liệu
df

,id,author,content,picture_count,processed,source,title,topic,url,crawled_at
0,218270,,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...",3,0,docbao.vn,"Tên cướp tiệm vàng tại Huế là đại uý công an, ...",Pháp luật,https://docbao.vn/phap-luat/ten-cuop-tiem-vang...,2022-08-01 09:09:22.817308
1,218269,(Nguồn: Sina),"Gần đây, Thứ trưởng Bộ Phát triển Kỹ thuật số,...",1,0,vtc.vn,"Bỏ qua mạng 5G, Nga tiến thẳng từ 4G lên 6G",Sống kết nối,https://vtc.vn/bo-qua-mang-5g-nga-tien-thang-t...,2022-08-01 09:09:21.181469
2,218268,Hồ Sỹ Anh,Kết quả thi tốt nghiệp THPT năm 2022 cho thấy ...,3,0,thanhnien.vn,Địa phương nào đứng đầu cả nước tổng điểm 3 mô...,Giáo dục,https://thanhnien.vn/dia-phuong-nao-dung-dau-c...,2022-08-01 09:09:15.311901
3,218267,Ngọc Ánh,Thống đốc Kentucky Andy Beshear hôm 31/7 cho h...,1,0,vnexpress,Người chết trong mưa lũ 'nghìn năm có một' ở M...,Thế giới,https://vnexpress.net/nguoi-chet-trong-mua-lu-...,2022-08-01 09:09:02.211498
4,218266,HẢI YẾN - MINH LÝ,Vụ tai nạn giao thông liên hoàn trên phố đi bộ...,12,0,soha,"Hải Phòng: Hình ảnh xe ""điên"" gây tai nạn liên...",Thời sự - Xã hội,https://soha.vn/hai-phong-hinh-anh-xe-dien-gay...,2022-08-01 09:09:01.601170
...,...,...,...,...,...,...,...,...,...,...
184534,5,Quốc Thắng,"Chiều 7/6, Khoa bị Công an quận Phú Nhuận phối...",2,1,vnexpress,Nghi can sát hại cha bị bắt,Pháp luật,https://vnexpress.net/nghi-can-sat-hai-cha-bi-...,2022-06-07 14:05:53.848085
184535,4,Thảo Mi,"Ngày 7/6, phiên xử phúc thẩm ông Tất Thành Can...",2,1,vnexpress,Ông Tất Thành Cang được đề nghị giảm án,Pháp luật,https://vnexpress.net/ong-tat-thanh-cang-duoc-...,2022-06-07 14:05:53.482540
184536,3,Thanh Tâm,"""Các khu dân cư của thành phố Severodonetsk đã...",2,1,vnexpress,Nga tuyên bố 'giải phóng' hoàn toàn khu dân cư...,Thế giới,https://vnexpress.net/nga-tuyen-bo-giai-phong-...,2022-06-07 14:05:51.797545
184537,2,Lâm Thỏa,"*Việt Nam - Malaysia: 20h thứ Tư 8/6, trên VnE...",2,1,vnexpress,HLV Malaysia dọa ngáng đường Việt Nam,Thể thao,https://vnexpress.net/hlv-malaysia-doa-ngang-d...,2022-06-07 14:05:51.433712


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184539 entries, 0 to 184538
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             184539 non-null  int64         
 1   author         184539 non-null  object        
 2   content        184539 non-null  object        
 3   picture_count  184539 non-null  int64         
 4   processed      184539 non-null  int64         
 5   source         184534 non-null  object        
 6   title          184539 non-null  object        
 7   topic          184539 non-null  object        
 8   url            184539 non-null  object        
 9   crawled_at     184125 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 14.1+ MB


In [6]:
df.columns

Index(['id', 'author', 'content', 'picture_count', 'processed', 'source',
       'title', 'topic', 'url', 'crawled_at'],
      dtype='object')

In [7]:
df = df[df['content'].apply(lambda x: not( str(x).isspace()  or len(str(x))<=20))]

# 3. Text Normalization

## 3.1 Lowercasing

In [8]:
df = df.copy()
df['content'] = df['content'].str.lower()
df['content'][0]

"chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo (tp huế, tỉnh thừa thiên - huế). thông sài gòn giải phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi (phía trước chợ đông ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên – huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

## 3.2 Extract and remove URLs

In [9]:
df = df.copy()

url_pattern = r'(http\S+|https\S+|www\S+)'
df['urls'] = df['content'].str.extractall(url_pattern).groupby(level=0).agg(lambda x: ', '.join(x))
df['content'] = df['content'].str.replace(url_pattern, '', regex=True)

In [10]:
df['content'][0]

"chiều 31/7, công an tỉnh thừa thiên - huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo (tp huế, tỉnh thừa thiên - huế). thông sài gòn giải phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi (phía trước chợ đông ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội, cách khu vực gây án khoảng 300m. giám đốc công an tỉnh thừa thiên – huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau nhặt, tạo cảnh nhốn

## 3.3 Remove punctuation characters

In [11]:
df = df.copy()

df['content'] = df['content'].str.translate(str.maketrans('', '', string.punctuation))

In [12]:
df['content'][0]

'chiều 317 công an tỉnh thừa thiên  huế đã có thông tin ban đầu về vụ nổ súngcướp tiệm vàng tại chợ đông ba nằm trên đường trần hưng đạo tp huế tỉnh thừa thiên  huế thông sài gòn giải phóng khoảng 12h30 ngày 317 một đối tượng sử dụng súng ak bất ngờ xông vào tiệm vàng hoàng đức và thái lợi phía trước chợ đông ba rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng sau đó đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu gia hội cách khu vực gây án khoảng 300m giám đốc công an tỉnh thừa thiên – huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm công an tỉnh và công an tp huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ khoanh vùng và ngăn không để người dân đi vào hiện trường hàng trăm tiểu thương trong chợ đông ba và người dân gần cầu gia hội được yêu cầu di chuyển khỏi hiện trường đóng cửa nhà đề phòng đạn lạc tuy nhiên thấy vàng bị ném ra đường nhiều người đua nhau nhặt tạo cảnh nhốn nháo trước cổng chợ do đâ

# 4. Text Preprocessing Techniques

## 4.1 Tokenizer

Tokenization refers to splitting raw text into smaller units, known as tokens. Tokens may represent words, subwords, or entire sentences, depending on the application.

In [13]:
df = df.copy()
df['tokenized_content'] = df['content'].apply(word_tokenize)


In [14]:
df_display = df[['content', 'tokenized_content']].head(5)
print(df_display)

                                             content  \
0  chiều 317 công an tỉnh thừa thiên  huế đã có t...   
1  gần đây thứ trưởng bộ phát triển kỹ thuật số t...   
2  kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...   
3  thống đốc kentucky andy beshear hôm 317 cho ha...   
4  vụ tai nạn giao thông liên hoàn trên phố đi bộ...   

                                   tokenized_content  
0  [chiều, 317, công, an, tỉnh, thừa, thiên, huế,...  
1  [gần, đây, thứ, trưởng, bộ, phát, triển, kỹ, t...  
2  [kết, quả, thi, tốt, nghiệp, thpt, năm, 2022, ...  
3  [thống, đốc, kentucky, andy, beshear, hôm, 317...  
4  [vụ, tai, nạn, giao, thông, liên, hoàn, trên, ...  


## 4.2  Stopword Removal

Stopwords are high-frequency words that usually provide little semantic value in text analysis. Removing them helps reduce dimensionality and noise in the data.  

For Vietnamese, a commonly used stopword dataset can be found here:  
👉 [Vietnamese Stopwords List](https://github.com/stopwords/vietnamese-stopwords/blob/master/vietnamese-stopwords.txt)

In [15]:
with open('/kaggle/input/vietnamese-stopwords/vietnamese-stopwords.txt', encoding='utf-8') as f:
    Vi_stopwords = {
        ''.join(ch for ch in line.strip() if not ch.isdigit())
        for line in f
        if line.strip() and not line.strip().isdigit()
    }

# Lọc bỏ stopwords trong dữ liệu
df['filter_stopwords'] = df['tokenized_content'].map(
    lambda tokens: [w for w in tokens if w not in Vi_stopwords]
)


In [16]:
df_display = df[['content', 'filter_stopwords']].head()
print(df_display)

                                             content  \
0  chiều 317 công an tỉnh thừa thiên  huế đã có t...   
1  gần đây thứ trưởng bộ phát triển kỹ thuật số t...   
2  kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...   
3  thống đốc kentucky andy beshear hôm 317 cho ha...   
4  vụ tai nạn giao thông liên hoàn trên phố đi bộ...   

                                    filter_stopwords  
0  [chiều, 317, công, an, tỉnh, thừa, thiên, huế,...  
1  [trưởng, phát, triển, kỹ, thuật, truyền, thông...  
2  [kết, thi, nghiệp, thpt, 2022, trung, bình, mô...  
3  [thống, đốc, kentucky, andy, beshear, hôm, 317...  
4  [vụ, tai, nạn, giao, thông, liên, hoàn, phố, đ...  


## 4.3 Bag of words

The **Bag-of-Words (BoW) model** is one of the most straightforward techniques for representing text in machine learning.  
It works by:  
1. Building a vocabulary of all unique words in the dataset.  
2. Representing each document as a vector of word counts.  

BoW ignores grammar and word order, focusing only on **word frequency**, which makes it simple but effective for many NLP tasks.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# Ghép token lại thành chuỗi vì CountVectorizer nhận string
df['clean_text'] = df['filter_stopwords'].apply(lambda x: ' '.join(x))

# Vector hóa
vectorizer = CountVectorizer(max_features=1000)
bow_matrix = vectorizer.fit_transform(df['clean_text'])

# DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_bow = pd.concat([df[['content']], bow_df], axis=1)

df_bow.head()


,content,10,100,11,12,13,14,15,16,17,...,ơn,ưu,ương,ước,ảnh,ấn,ổn,ủng,ủy,ứng
0,chiều 317 công an tỉnh thừa thiên huế đã có t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,gần đây thứ trưởng bộ phát triển kỹ thuật số t...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...,6.0,0.0,4.0,4.0,4.0,5.0,8.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,thống đốc kentucky andy beshear hôm 317 cho ha...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,vụ tai nạn giao thông liên hoàn trên phố đi bộ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4.4 TF-IDF

**TF-IDF** is a text representation technique that improves on Bag-of-Words by weighting words based on importance.  
- **Term Frequency (TF):** How often a word appears in a document.  
- **Inverse Document Frequency (IDF):** How unique or rare a word is across all documents.  

The combination highlights informative words while reducing the influence of very common ones.  

In [18]:
df = df.copy()

df['clean_text'] = df['filter_stopwords'].apply(lambda x: ' '.join(x))

# TF-IDF
tfidf = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf.fit_transform(df['clean_text'])

# DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())
df_tfidf = pd.concat([df[['content']], tfidf_df], axis=1)

df_tfidf.head()

,content,10,100,11,12,13,14,15,16,17,...,ơn,ưu,ương,ước,ảnh,ấn,ổn,ủng,ủy,ứng
0,chiều 317 công an tỉnh thừa thiên huế đã có t...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,gần đây thứ trưởng bộ phát triển kỹ thuật số t...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,kết quả thi tốt nghiệp thpt năm 2022 cho thấy ...,0.071627,0.000000,0.063922,0.058137,0.06926,0.088544,0.119823,0.070303,0.017104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.014669,0.0,0.0,0.0
3,thống đốc kentucky andy beshear hôm 317 cho ha...,0.000000,0.063964,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,vụ tai nạn giao thông liên hoàn trên phố đi bộ...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [19]:
# ---- Lấy top 10 từ quan trọng nhất ----
tfidf_sum = tfidf_matrix.sum(axis=0).A1   # tổng TF-IDF của mỗi từ
tfidf_freq = dict(zip(tfidf.get_feature_names_out(), tfidf_sum))

top_n = 10
top_tfidf_terms = sorted(tfidf_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]
terms, scores = zip(*top_tfidf_terms)

# ---- Vẽ biểu đồ ----
from wordcloud import WordCloud

wordcloud = WordCloud(width=800, height=400, background_color='white',
                      colormap='Set2').generate_from_frequencies(dict(top_tfidf_terms))

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Top 10 Terms by TF-IDF (WordCloud)')
plt.show()


NameError: name 'plt' is not defined

In [ ]:
colors = cm.viridis(np.linspace(0, 1, len(terms)))  # chọn bảng màu viridis

plt.figure(figsize=(10, 5))
plt.barh(terms, scores, color=colors)
plt.xlabel('TF-IDF Score')
plt.ylabel('Terms')
plt.title('Top 10 Terms by TF-IDF (Colormap)')
plt.xlim(0, max(scores) * 1.1)
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

## 4.5 Unigram

In addition to the basic TF-IDF representation, we can restrict the feature extraction to **unigrams** (single words).  
This technique helps us capture the importance of each word in the text corpus without considering word pairs or longer n-grams.

In [ ]:
# # TF-IDF + unigram
# tfidf_uni = TfidfVectorizer(ngram_range=(1,1), max_features=1000)
# tfidf_uni_matrix = tfidf_uni.fit_transform(df['content'])

# tfidf_uni_df = pd.DataFrame(tfidf_uni_matrix.toarray(), columns=tfidf_uni.get_feature_names_out())

# df_tfidf_uni = pd.concat([df[['content']], tfidf_uni_df], axis=1)
# df_tfidf_uni.head()


In [ ]:
# # Tổng TF-IDF của từng từ
# tfidf_sum = tfidf_uni_matrix.sum(axis=0).A1
# tfidf_freq = dict(zip(tfidf_uni.get_feature_names_out(), tfidf_sum))

# top_n = 20
# top_terms = sorted(tfidf_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]
# terms, scores = zip(*top_terms)

# # Vẽ biểu đồ
# plt.figure(figsize=(12,6))
# sns.barplot(x=list(scores), y=list(terms), palette="viridis")
# plt.xlabel("Tổng TF-IDF")
# plt.ylabel("Từ")
# plt.title("Top 20 từ theo TF-IDF")
# plt.show()